###  LabelEncoder和OneHotEncoder
+ https://www.jianshu.com/p/91ef6b733885

In [4]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder() 
le.fit([1, 2, 2, 6])
# le.classes_ 
# le.transform([1, 1, 2, 6]) 
le.inverse_transform([0, 0, 1, 2])

array([1, 1, 2, 6])

In [10]:
le = preprocessing.LabelEncoder() 
le.fit(["paris", "paris", "tokyo", "amsterdam"]) 
# list(le.classes_)
list(le.inverse_transform([2, 2, 1]))
# le.inverse_transform([2, 2, 1])
# le.transform(["paris", "paris", "tokyo", "amsterdam"])

['tokyo', 'tokyo', 'paris']

In [19]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit([[1],[2],[3],[4]])
ohe.transform([[2],[3],[1],[4]]).toarray() 

H:\Anaconda3\anzhuang1\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [38]:
from sklearn import preprocessing 
enc=preprocessing.OneHotEncoder() 
a = ([[0,0,3],[1,1,0],[0,2,1],[1,0,2]])    # 第一列只有0和1，第二列有0,1,2三种，第三列有0,1,2,3四类，[]中前两个、后三个、最后四个分别表示这三列
print(a)
enc.fit(a) 
array=enc.transform([[0,1,3]]).toarray() 
print(array)
# enc.transform([[0,1,3]]).toarray() 
b = enc.transform([[1,1,3]]).toarray() 
c = enc.transform([[0,2,3]]).toarray() 
d = enc.transform([[0,1,2]]).toarray() 
print(b)
print(c)
print(d)
print('- '*15)
e = enc.transform([[0,1,3],[0,2,3]]).toarray() 
print(e)

[[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]]
[[1. 0. 0. 1. 0. 0. 0. 0. 1.]]
[[0. 1. 0. 1. 0. 0. 0. 0. 1.]]
[[1. 0. 0. 0. 1. 0. 0. 0. 1.]]
[[1. 0. 0. 1. 0. 0. 0. 1. 0.]]
- - - - - - - - - - - - - - - 
[[1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1.]]


H:\Anaconda3\anzhuang1\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## 导入包、读取数据

In [1]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score

In [2]:
path='./'
train = pd.read_csv(path+'input/train_set.csv')
test = pd.read_csv(path+'input/test_set.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
ID           25317 non-null int64
age          25317 non-null int64
job          25317 non-null object
marital      25317 non-null object
education    25317 non-null object
default      25317 non-null object
balance      25317 non-null int64
housing      25317 non-null object
loan         25317 non-null object
contact      25317 non-null object
day          25317 non-null int64
month        25317 non-null object
duration     25317 non-null int64
campaign     25317 non-null int64
pdays        25317 non-null int64
previous     25317 non-null int64
poutcome     25317 non-null object
y            25317 non-null int64
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 17 columns):
ID           10852 non-null int64
age          10852 non-null int64
job          10852 non-null object
mar

In [3]:
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [4]:
test.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000
mean,30743.500000,41.040638,1393.157298,15.680151,257.206137,2.770365,39.954755,0.546443
std,3132.846895,10.652369,3318.497101,8.302317,250.480906,3.063481,99.524056,1.805938
min,25318.000000,18.000000,-2604.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,28030.750000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,30743.500000,39.000000,450.000000,16.000000,181.000000,2.000000,-1.000000,0.000000
75%,33456.250000,49.000000,1440.250000,21.000000,322.000000,3.000000,-1.000000,0.000000
max,36169.000000,94.000000,81204.000000,31.000000,3102.000000,58.000000,871.000000,51.000000


In [5]:
train.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


## 数据预处理

In [6]:
train.job.value_counts()

# value_counts  https://www.jianshu.com/p/f773b4b82c66  查看表格某列中有多少个不同值的快捷方法，并计算每个不同值在该列中有多少重复值。

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [7]:
test['y']=-1
print(len(test.columns))

18


In [8]:
# print(test.head())

In [9]:
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).reset_index(drop=True)
print(data.head())

   ID  age         job   marital  education default  balance housing loan  \
0   1   43  management   married   tertiary      no      291     yes   no   
1   2   42  technician  divorced    primary      no     5076     yes   no   
2   3   47      admin.   married  secondary      no      104     yes  yes   
3   4   28  management    single  secondary      no     -994     yes  yes   
4   5   42  technician  divorced  secondary      no     2974     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0   unknown    9   may       150         2     -1         0  unknown  0  
1  cellular    7   apr        99         1    251         2    other  0  
2  cellular   14   jul        77         2     -1         0  unknown  0  
3  cellular   18   jul       174         2     -1         0  unknown  0  
4   unknown   21   may       187         5     -1         0  unknown  0  


In [10]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

## 构建特征

In [11]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()     # 类别型特征转化为数值的
    data['count_' + i] = data.groupby([i])[i].transform('count')   # count特征  ？？？？
    data[i] = lbl.fit_transform(data[i].astype(str))

In [9]:
import pandas as pd
 
df = pd.DataFrame({'Country':['China','China', 'India', 'India', 'America', 'Japan', 'China', 'India'], 
                   'Income':[10000, 10000, 5000, 5002, 40000, 50000, 8000, 5000],
                    'Age':[5000, 4321, 1234, 4010, 250, 250, 4500, 4321]})
df

,Country,Income,Age
0,China,10000,5000
1,China,10000,4321
2,India,5000,1234
3,India,5002,4010
4,America,40000,250
5,Japan,50000,250
6,China,8000,4500
7,India,5000,4321


In [12]:
df['count_' + 'Country'] = df.groupby('Country')['Country'].transform('count') 
df['count_' + 'Country']

0    3
1    3
2    3
3    3
4    1
5    1
6    3
7    3
Name: count_Country, dtype: int64

1. 根据数据类型选择特征（此处为对象特征），生成列表
        https://www.cnblogs.com/liyun1/p/11267828.html
        https://blog.csdn.net/qq_37277944/article/details/82985167#%E5%9B%9Bselect_dtypes%EF%BC%9A
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html
2. 按([i])i列聚类，并对[i]i列元素计数，并赋值给新列   ----i在此处意义不大？？？
        https://blog.csdn.net/gaozhanfire/article/details/95663716
        https://www.cnblogs.com/Yanjy-OnlyOne/p/11217802.html
3. 类别编号（如job分别编号为0、1、。。。）
        https://www.cnblogs.com/sench/p/10134094.html

In [12]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,...,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,...,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,...,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,...,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,...,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


1ID Int客户唯一标识

2age Int客户年龄

3job String客户的职业

4marital String婚姻状况

5education String受教育水平

6default String是否有违约记录

7balance Int每年账户的平均余额

8housing String是否有住房贷款

9loan String是否有个人贷款

10contact String与客户联系的沟通方式

11day Int最后一次联系的时间（几号）

12month String最后一次联系的时间（月份）

13duration Int最后一次联系的交流时长

14campaign Int在本次活动中，与该客户交流过的次数

15pdays Int距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过）

16previousInt在本次活动之前，与该客户交流过的次数

17poutcome String上一次活动的结果

18y Int预测客户是否会订购定期存款业务

In [13]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,...,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,...,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,...,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,...,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,...,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


In [14]:
feats = [i for i in data.columns if i not in ['ID','y']]
feats

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'count_job',
 'count_marital',
 'count_education',
 'count_default',
 'count_housing',
 'count_loan',
 'count_contact',
 'count_month',
 'count_poutcome']

## 训练模型及预测

In [15]:
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )
train_x =data[data['y']!=-1][feats]
train_y =data[data['y']!=-1]['y']
testx= data[data['y']==-1][feats]
# train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)    # 不切割成训练集和测试集两部分，直接用全局来训练模型
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration # 第二列

In [18]:
test_pre

array([0.03482698, 0.0055288 , 0.00122388, ..., 0.00095247, 0.00143075,
       0.03868457])

In [19]:
print(len(test_pre))

10852


In [21]:
# type(data[data['y']==-1]['ID'])

In [22]:
# type(data[data['y']==-1][['ID']])

In [23]:
pre=data[data['y']==-1][['ID']]
pre['pred']=test_pre
print(len(pre))

10852


In [24]:
pre.head()

,ID,pred
25317,25318,0.034827
25318,25319,0.005529
25319,25320,0.001224
25320,25321,0.746336
25321,25322,0.016578


In [25]:
pre.to_csv('baseline2.csv',index=False )

In [26]:
check = pd.read_csv(path+'baseline2.csv')
print(check.head())

      ID      pred
0  25318  0.034827
1  25319  0.005529
2  25320  0.001224
3  25321  0.746336
4  25322  0.016578


## 采用K-fold方式，划分训练集和测试集，来训练模型，预测

In [27]:
#10折交叉验证
from sklearn.model_selection import KFold
n_splits=10
kfold = KFold(n_splits=10, shuffle=True, random_state=42)  # shuffle是否打乱，如果特征的时序性比较强，不要打乱
train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']
res=data[data['y']==-1][['ID']]     # dataframe类型

test_x= data[data['y']==-1][feats]
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)   # 采用上述lgb模型
    res['pred'] += model.predict_proba(test_x)[:,1]   # 第二列

res['pred'] = res['pred']/10
res.to_csv('./submission10_fold.csv', index=False)

[1]	training's auc: 0.76357	valid_1's auc: 0.763269
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.907211	valid_1's auc: 0.894519
[3]	training's auc: 0.907136	valid_1's auc: 0.896358
[4]	training's auc: 0.903751	valid_1's auc: 0.895236
[5]	training's auc: 0.91492	valid_1's auc: 0.902974
[6]	training's auc: 0.919756	valid_1's auc: 0.910151
[7]	training's auc: 0.92256	valid_1's auc: 0.914946
[8]	training's auc: 0.925151	valid_1's auc: 0.916957
[9]	training's auc: 0.924994	valid_1's auc: 0.915093
[10]	training's auc: 0.923731	valid_1's auc: 0.913914
[11]	training's auc: 0.927422	valid_1's auc: 0.917375
[12]	training's auc: 0.930098	valid_1's auc: 0.919177
[13]	training's auc: 0.931007	valid_1's auc: 0.9194
[14]	training's auc: 0.930158	valid_1's auc: 0.917978
[15]	training's auc: 0.931003	valid_1's auc: 0.918974
[16]	training's auc: 0.930201	valid_1's auc: 0.91735
[17]	training's auc: 0.931179	valid_1's auc: 0.918352
[18]	training's auc: 0.931777	valid

[300]	training's auc: 0.961307	valid_1's auc: 0.932913
[301]	training's auc: 0.961373	valid_1's auc: 0.93296
[302]	training's auc: 0.961452	valid_1's auc: 0.932963
[303]	training's auc: 0.961535	valid_1's auc: 0.932927
[304]	training's auc: 0.961587	valid_1's auc: 0.932861
[305]	training's auc: 0.961653	valid_1's auc: 0.932855
[306]	training's auc: 0.961713	valid_1's auc: 0.932869
[307]	training's auc: 0.961807	valid_1's auc: 0.932883
[308]	training's auc: 0.961894	valid_1's auc: 0.932886
[309]	training's auc: 0.961965	valid_1's auc: 0.932961
[310]	training's auc: 0.962044	valid_1's auc: 0.933007
[311]	training's auc: 0.962137	valid_1's auc: 0.933078
[312]	training's auc: 0.962196	valid_1's auc: 0.933078
[313]	training's auc: 0.96229	valid_1's auc: 0.933142
[314]	training's auc: 0.962351	valid_1's auc: 0.933153
[315]	training's auc: 0.962435	valid_1's auc: 0.933061
[316]	training's auc: 0.962497	valid_1's auc: 0.933098
[317]	training's auc: 0.962555	valid_1's auc: 0.933101
[318]	traini

[45]	training's auc: 0.938668	valid_1's auc: 0.93607
[46]	training's auc: 0.938709	valid_1's auc: 0.936139
[47]	training's auc: 0.938863	valid_1's auc: 0.9362
[48]	training's auc: 0.938892	valid_1's auc: 0.93617
[49]	training's auc: 0.938938	valid_1's auc: 0.936307
[50]	training's auc: 0.938995	valid_1's auc: 0.936409
[51]	training's auc: 0.939046	valid_1's auc: 0.936468
[52]	training's auc: 0.939178	valid_1's auc: 0.936456
[53]	training's auc: 0.939254	valid_1's auc: 0.936454
[54]	training's auc: 0.939396	valid_1's auc: 0.936468
[55]	training's auc: 0.939466	valid_1's auc: 0.936425
[56]	training's auc: 0.939739	valid_1's auc: 0.936639
[57]	training's auc: 0.939747	valid_1's auc: 0.936606
[58]	training's auc: 0.939895	valid_1's auc: 0.936557
[59]	training's auc: 0.939968	valid_1's auc: 0.936605
[60]	training's auc: 0.940072	valid_1's auc: 0.936505
[61]	training's auc: 0.940117	valid_1's auc: 0.936384
[62]	training's auc: 0.940253	valid_1's auc: 0.93638
[63]	training's auc: 0.940288	val

[344]	training's auc: 0.963678	valid_1's auc: 0.942263
[345]	training's auc: 0.963765	valid_1's auc: 0.942243
[346]	training's auc: 0.963825	valid_1's auc: 0.942249
[347]	training's auc: 0.963896	valid_1's auc: 0.942224
[348]	training's auc: 0.963954	valid_1's auc: 0.942198
[349]	training's auc: 0.964017	valid_1's auc: 0.942208
[350]	training's auc: 0.964052	valid_1's auc: 0.94217
[351]	training's auc: 0.964118	valid_1's auc: 0.942187
[352]	training's auc: 0.96417	valid_1's auc: 0.942145
[353]	training's auc: 0.964231	valid_1's auc: 0.942125
[354]	training's auc: 0.96429	valid_1's auc: 0.942118
[355]	training's auc: 0.96434	valid_1's auc: 0.942089
[356]	training's auc: 0.964416	valid_1's auc: 0.942086
[357]	training's auc: 0.964485	valid_1's auc: 0.942094
[358]	training's auc: 0.964549	valid_1's auc: 0.942083
[359]	training's auc: 0.964597	valid_1's auc: 0.942134
[360]	training's auc: 0.964653	valid_1's auc: 0.942149
[361]	training's auc: 0.964763	valid_1's auc: 0.942149
[362]	training

[201]	training's auc: 0.954445	valid_1's auc: 0.934693
[202]	training's auc: 0.954505	valid_1's auc: 0.934725
[203]	training's auc: 0.954576	valid_1's auc: 0.934766
[204]	training's auc: 0.954644	valid_1's auc: 0.934745
[205]	training's auc: 0.954742	valid_1's auc: 0.93482
[206]	training's auc: 0.954823	valid_1's auc: 0.934898
[207]	training's auc: 0.95487	valid_1's auc: 0.934892
[208]	training's auc: 0.954936	valid_1's auc: 0.934928
[209]	training's auc: 0.955006	valid_1's auc: 0.934869
[210]	training's auc: 0.955072	valid_1's auc: 0.934932
[211]	training's auc: 0.955139	valid_1's auc: 0.934919
[212]	training's auc: 0.955204	valid_1's auc: 0.934987
[213]	training's auc: 0.955264	valid_1's auc: 0.935034
[214]	training's auc: 0.955336	valid_1's auc: 0.935071
[215]	training's auc: 0.955439	valid_1's auc: 0.935103
[216]	training's auc: 0.955486	valid_1's auc: 0.935136
[217]	training's auc: 0.955559	valid_1's auc: 0.935187
[218]	training's auc: 0.955644	valid_1's auc: 0.935188
[219]	traini

...

[625]	training's auc: 0.977036	valid_1's auc: 0.936198
[626]	training's auc: 0.977062	valid_1's auc: 0.936219
[627]	training's auc: 0.977095	valid_1's auc: 0.936231
[628]	training's auc: 0.977139	valid_1's auc: 0.936254
[629]	training's auc: 0.977164	valid_1's auc: 0.936274
[630]	training's auc: 0.977186	valid_1's auc: 0.936297
[631]	training's auc: 0.977224	valid_1's auc: 0.936302
[632]	training's auc: 0.977258	valid_1's auc: 0.936321
[633]	training's auc: 0.977299	valid_1's auc: 0.936331
[634]	training's auc: 0.977339	valid_1's auc: 0.936351
[635]	training's auc: 0.977377	valid_1's auc: 0.93635
[636]	training's auc: 0.977405	valid_1's auc: 0.936363
[637]	training's auc: 0.977428	valid_1's auc: 0.936356
[638]	training's auc: 0.977467	valid_1's auc: 0.936366
[639]	training's auc: 0.9775	valid_1's auc: 0.936326
[640]	training's auc: 0.977541	valid_1's auc: 0.936328
[641]	training's auc: 0.977562	valid_1's auc: 0.936317
[642]	training's auc: 0.977615	valid_1's auc: 0.936308
[643]	trainin

[94]	training's auc: 0.944428	valid_1's auc: 0.931422
[95]	training's auc: 0.944489	valid_1's auc: 0.931415
[96]	training's auc: 0.944597	valid_1's auc: 0.931391
[97]	training's auc: 0.944713	valid_1's auc: 0.931478
[98]	training's auc: 0.944821	valid_1's auc: 0.931474
[99]	training's auc: 0.94489	valid_1's auc: 0.931485
[100]	training's auc: 0.944983	valid_1's auc: 0.931389
[101]	training's auc: 0.945071	valid_1's auc: 0.931491
[102]	training's auc: 0.945157	valid_1's auc: 0.93145
[103]	training's auc: 0.945292	valid_1's auc: 0.931477
[104]	training's auc: 0.9455	valid_1's auc: 0.931696
[105]	training's auc: 0.945565	valid_1's auc: 0.93177
[106]	training's auc: 0.945657	valid_1's auc: 0.931874
[107]	training's auc: 0.945801	valid_1's auc: 0.931899
[108]	training's auc: 0.94597	valid_1's auc: 0.932059
[109]	training's auc: 0.946049	valid_1's auc: 0.932137
[110]	training's auc: 0.946123	valid_1's auc: 0.93207
[111]	training's auc: 0.946213	valid_1's auc: 0.932129
[112]	training's auc: 0

[392]	training's auc: 0.966495	valid_1's auc: 0.938294
[393]	training's auc: 0.966536	valid_1's auc: 0.938289
[394]	training's auc: 0.966573	valid_1's auc: 0.938291
[395]	training's auc: 0.966626	valid_1's auc: 0.938228
[396]	training's auc: 0.966677	valid_1's auc: 0.938208
[397]	training's auc: 0.966724	valid_1's auc: 0.938209
[398]	training's auc: 0.966807	valid_1's auc: 0.938246
[399]	training's auc: 0.966858	valid_1's auc: 0.938274
[400]	training's auc: 0.966906	valid_1's auc: 0.938252
[401]	training's auc: 0.966974	valid_1's auc: 0.938333
[402]	training's auc: 0.967034	valid_1's auc: 0.93831
[403]	training's auc: 0.967084	valid_1's auc: 0.938302
[404]	training's auc: 0.967138	valid_1's auc: 0.938341
[405]	training's auc: 0.967201	valid_1's auc: 0.938358
[406]	training's auc: 0.967234	valid_1's auc: 0.93833
[407]	training's auc: 0.967289	valid_1's auc: 0.938353
[408]	training's auc: 0.967348	valid_1's auc: 0.938369
[409]	training's auc: 0.967405	valid_1's auc: 0.938367
[410]	traini

[122]	training's auc: 0.949481	valid_1's auc: 0.922478
[123]	training's auc: 0.949534	valid_1's auc: 0.922398
[124]	training's auc: 0.949622	valid_1's auc: 0.92246
[125]	training's auc: 0.949746	valid_1's auc: 0.922432
[126]	training's auc: 0.949802	valid_1's auc: 0.922321
[127]	training's auc: 0.94989	valid_1's auc: 0.922503
[128]	training's auc: 0.949963	valid_1's auc: 0.922503
[129]	training's auc: 0.95005	valid_1's auc: 0.922657
[130]	training's auc: 0.950188	valid_1's auc: 0.922744
[131]	training's auc: 0.950293	valid_1's auc: 0.92275
[132]	training's auc: 0.950331	valid_1's auc: 0.922689
[133]	training's auc: 0.950427	valid_1's auc: 0.922824
[134]	training's auc: 0.950487	valid_1's auc: 0.922845
[135]	training's auc: 0.950572	valid_1's auc: 0.923024
[136]	training's auc: 0.950625	valid_1's auc: 0.923094
[137]	training's auc: 0.950731	valid_1's auc: 0.923201
[138]	training's auc: 0.950785	valid_1's auc: 0.923096
[139]	training's auc: 0.950852	valid_1's auc: 0.923204
[140]	training

[420]	training's auc: 0.969369	valid_1's auc: 0.925377
[421]	training's auc: 0.969416	valid_1's auc: 0.925373
[422]	training's auc: 0.969445	valid_1's auc: 0.925361
[423]	training's auc: 0.969514	valid_1's auc: 0.925404
[424]	training's auc: 0.96956	valid_1's auc: 0.925397
[425]	training's auc: 0.969614	valid_1's auc: 0.925343
[426]	training's auc: 0.969667	valid_1's auc: 0.925318
[427]	training's auc: 0.969724	valid_1's auc: 0.925319
[428]	training's auc: 0.969781	valid_1's auc: 0.925326
[429]	training's auc: 0.969824	valid_1's auc: 0.925298
[430]	training's auc: 0.969885	valid_1's auc: 0.925356
[431]	training's auc: 0.969921	valid_1's auc: 0.925366
[432]	training's auc: 0.969954	valid_1's auc: 0.92535
[433]	training's auc: 0.969987	valid_1's auc: 0.925356
[434]	training's auc: 0.970031	valid_1's auc: 0.925426
[435]	training's auc: 0.97008	valid_1's auc: 0.925385
[436]	training's auc: 0.970143	valid_1's auc: 0.925354
[437]	training's auc: 0.970217	valid_1's auc: 0.925289
[438]	trainin